In [1]:
!pip install -q bitsandbytes accelerate datasets peft transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!nvidia-smi

Mon Apr 21 17:36:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import pandas as pd
import json
import re
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
from peft import get_peft_model, LoraConfig, TaskType
import torch

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
dataset = load_dataset("json", data_files="/content/drive/MyDrive/formal_informal_pairs.jsonl")["train"]

def clean_quotes(example):
    for field in ["formal", "informal"]:
        example[field] = re.sub(r'^["\']+|["\']+$', '', example[field].strip())
    return example

dataset = dataset.map(clean_quotes)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
from huggingface_hub import login

login("token")

model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
def format_example(example):
    return f"### Instruction:\nMake this sentence informal.\n\n### Input:\n{example['formal']}\n\n### Response:\n{example['informal']}"

def tokenize(example):
    return tokenizer(
        format_example(example),
        padding="max_length",
        truncation=True,
        max_length=512
    )

In [8]:
tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
from transformers import TrainerCallback

class MetricsLoggerCallback(TrainerCallback):
    def __init__(self):
        self.epoch_logs = []

    def on_epoch_end(self, args, state, control, **kwargs):
        log = {
            "epoch": int(state.epoch),
            "loss": round(state.log_history[-1]["loss"], 4),
            "learning_rate": round(state.log_history[-1].get("learning_rate", 0), 8),
            "step": int(state.global_step),
            "time": round(time.time(), 2)
        }
        print(f"Epoch {log['epoch']} | Loss: {log['loss']}")
        self.epoch_logs.append(log)

metrics_logger = MetricsLoggerCallback()

In [12]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/mistral-style-transfer",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none",
    resume_from_checkpoint=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    callbacks=[metrics_logger]
)

<ipython-input-12-7e09d1ed402c>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [13]:
import torch
torch.cuda.empty_cache()

In [16]:
import os
from transformers.trainer_utils import get_last_checkpoint

checkpoint_dir = get_last_checkpoint(training_args.output_dir)

if checkpoint_dir is not None:
    print(f"Resuming from checkpoint: {checkpoint_dir}")
    trainer.train(resume_from_checkpoint=checkpoint_dir)
else:
    print("No checkpoint found — starting fresh.")
    trainer.train()

# save metrics
metrics_df = pd.DataFrame(metrics_logger.epoch_logs)
metrics_df.to_csv("mistral_training_metrics.csv", index=False)
metrics_df.to_csv("/content/drive/MyDrive/mistral_training_metrics.csv", index=False)
print("Training metrics saved to mistral_training_metrics.csv")

# save model
model.save_pretrained("/content/drive/MyDrive/mistral-style-transfer-lora")
tokenizer.save_pretrained("/content/drive/MyDrive/mistral-style-transfer-lora")
print("Model saved")

model = model.merge_and_unload()
model.save_pretrained("/content/drive/MyDrive/mistral-style-transfer-merged", safe_serialization=True)
tokenizer.save_pretrained("/content/drive/MyDrive/mistral-style-transfer-merged")
print("Merged model saved")

Resuming from checkpoint: /content/drive/MyDrive/mistral-style-transfer/checkpoint-936


Step,Training Loss


Training metrics saved to mistral_training_metrics.csv
Model saved


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Merged model saved


In [17]:
import gc
import torch

# Clear CUDA cache
gc.collect()
torch.cuda.empty_cache()

# Optionally delete old model
del model

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer

inference_path = "/content/drive/MyDrive/mistral-style-transfer-merged"

tokenizer = AutoTokenizer.from_pretrained(inference_path)
model = AutoModelForCausalLM.from_pretrained(
    inference_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [29]:
# Inference example
input_text = "### Instruction:\nMake this sentence informal.\n\n### Input:\nI do not know the answer.\n\n### Response:\n"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=20,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        repetition_penalty=1.2
    )

output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output.split('###')[3].split('Response:')[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



I'm not sure.




In [19]:
import math
from torch.nn import CrossEntropyLoss

def calculate_perplexity(model, tokenizer, text):
    encodings = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    input_ids = encodings.input_ids
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
    return math.exp(loss.item())

# Example usage
sample_text = "### Instruction:\nMake this sentence informal.\n\n### Input:\nI do not know the answer.\n\n### Response:\nI dunno the answer."
ppl = calculate_perplexity(model, tokenizer, sample_text)
print(f"Perplexity: {ppl:.2f}")

Perplexity: 2.44
